In [1]:
# make our imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

census_canada_df = pd.read_csv("./csv/census_canada_scrape.csv")
# get the cities we want from the census data
cities = census_canada_df['City'].unique().tolist()

# base url where we append all the city as a param
base_url = f"https://www.walkscore.com/score"


# for each city try to get the walk score
for city in cities:
    
    # get the page and parse it
    page = requests.get(f"{base_url}/{city}-ontario")
    soup = BeautifulSoup(page.content, "html.parser")
    
    # if the walk score exists, get it or else set it to 0
    try:
        walk_score = (
            soup.find("div", class_="block-header-badge score-info-link")
            .find("img", alt=True)
            .get("alt")
            .split(" ")[0]
        )
    except:
        walk_score = 0
        
    # if the bike score exists, get it or else set it to 0
    try:
        bike_score = soup.find_all("div", {"data-type": "bike"})[0].find("img").get("alt").split(" ")[0]
    except:
        bike_score = 0
        
    # if the transit score exists, get it or else set it to 0
    try:
        transit_score = soup.find_all("div", {"data-type": "bike"})[0].find("img").get("alt").split(" ")[0]
    except:
        transit_score = 0
    
    # set the walk score, bike score, and transit score    
    census_canada_df.loc[census_canada_df['City'] == city, 'walk score'] = walk_score
    census_canada_df.loc[census_canada_df['City'] == city, 'bike score'] = bike_score
    census_canada_df.loc[census_canada_df['City'] == city, 'transit score'] = transit_score    

In [3]:
# save to csv
census_canada_df.to_csv('./csv/census_canada_scrape.csv')
